# Execution

This notebook explains how PartiQL executes, with some discussion of its implementation.

## Data model

As discussed in [02-data-model.ipynb](02-data-model.ipynb), the only allowed types are PLURP (PLUR in this demo). In a rowwise, dynamically typed interpreter, this means there are only values, records, and lists. Since PartiQL is list order-independent and merges items with the same key, the lists are really sets, though the word "list" appears in the implementation and error messages.

Every value entering or exiting the PartiQL interpreter is a `data.Instance`, even simple numbers.

In [3]:
import data

arrays = data.RecordArray({
    "x": data.PrimitiveArray([0.1, 0.2, 0.3]),
    "y": data.PrimitiveArray([1000, 2000, 3000]),
    "table1": data.ListArray([0, 3, 3], [3, 3, 5], data.RecordArray({
            "a": data.PrimitiveArray([1.1, 2.2, 3.3, 4.4, 5.5]),
            "b": data.PrimitiveArray([100, 200, 300, 400, 500])
        })
    ),
    "table2": data.ListArray([0, 2, 3], [2, 3, 8], data.RecordArray({
            "b": data.PrimitiveArray([True, False, True, False, False, True, True, True]),
            "c": data.PrimitiveArray([10, 20, 100, 1, 2, 3, 4, 5])
        })
    )
})

arrays.tolist()

[{'x': 0.1,
  'y': 1000,
  'table1': [{'a': 1.1, 'b': 100}, {'a': 2.2, 'b': 200}, {'a': 3.3, 'b': 300}],
  'table2': [{'b': True, 'c': 10}, {'b': False, 'c': 20}]},
 {'x': 0.2, 'y': 2000, 'table1': [], 'table2': [{'b': True, 'c': 100}]},
 {'x': 0.3,
  'y': 3000,
  'table1': [{'a': 4.4, 'b': 400}, {'a': 5.5, 'b': 500}],
  'table2': [{'b': False, 'c': 1},
   {'b': False, 'c': 2},
   {'b': True, 'c': 3},
   {'b': True, 'c': 4},
   {'b': True, 'c': 5}]}]

In [5]:
arrays.setindex()

instances = data.instantiate(arrays)
instances

List{ 
    Rec#0(0)(){ 
        x = Value#0(0)('x'){ 0.1 }
        y = Value#0(0)('y'){ 1000 }
        table1 = List#0(0)('table1'){ 
            Rec#1(0 0)('table1'){ 
                a = Value#1(0 0)('table1' 'a'){ 1.1 }
                b = Value#1(0 0)('table1' 'b'){ 100 }
            }
            Rec#1(0 1)('table1'){ 
                a = Value#1(0 1)('table1' 'a'){ 2.2 }
                b = Value#1(0 1)('table1' 'b'){ 200 }
            }
            Rec#1(0 2)('table1'){ 
                a = Value#1(0 2)('table1' 'a'){ 3.3 }
                b = Value#1(0 2)('table1' 'b'){ 300 }
            }
        }
        table2 = List#0(0)('table2'){ 
            Rec#2(0 0)('table2'){ 
                b = Value#2(0 0)('table2' 'b'){ True }
                c = Value#2(0 0)('table2' 'c'){ 10 }
            }
            Rec#2(0 1)('table2'){ 
                b = Value#2(0 1)('table2' 'b'){ False }
                c = Value#2(0 1)('table2' 'c'){ 20 }
            }
        }
    }
    Rec#0(1)(){

## Inputs and outputs

Next, we'll run a few simple examples to show what the runtime engine requires and produces.

If you have not already done so, install the [Lark parser](https://github.com/lark-parser/lark#readme) and Matplotlib.

In [6]:
!pip install lark-parser matplotlib